<a href="https://colab.research.google.com/github/jzemgulys/plant_leaf_model1/blob/main/Leaf_Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import os
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import pandas as pd
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
root = 'drive/MyDrive/leaf_classif'

In [ ]:
# NOTE: you might have to "Add shortcut to Drive" in "shared with you" on google drive
#       if it cannot find the folder in main root "drive" directory
files = [file for file in os.listdir(root) if os.path.isfile(os.path.join(root, file))]
# or however we process the files in the shared dir.

In [ ]:
!pwd

/content


In [ ]:
main_dir = root  # Replace with your main directory

def label_from_folder_name(folder_name):
    if 'healthy' in folder_name:
        return 'healthy'  # Class label for healthy images
    elif 'diseased' in folder_name:
        return 'diseased'  # Class label for unhealthy images
    else:
        return None  # No specific label found

def custom_flow_from_directory(directory, target_size, batch_size):
    filenames = []
    labels = []

    for folder in os.listdir(directory):
        folder_path = os.path.join(directory, folder)
        if os.path.isdir(folder_path):
            for root, dirs, files in os.walk(folder_path):
                for file in files:
                    filenames.append(os.path.join(root, file))
                    labels.append(label_from_folder_name(root))

    filenames = np.array(filenames)
    labels = np.array(labels, dtype=str)  # Ensure labels are strings

    return ImageDataGenerator(rescale=1./255).flow_from_dataframe(
        pd.DataFrame({"filename": filenames, "class": labels}),
        x_col="filename",
        y_col="class",
        target_size=target_size,
        batch_size=batch_size,
        class_mode='binary'
    )

# Use the custom function to load data
batch_size = 32
img_height, img_width = 150, 150

train_dir = os.path.join(main_dir, 'train')
test_dir = os.path.join(main_dir, 'test')

train_generator = custom_flow_from_directory(train_dir, (img_height, img_width), batch_size)
test_generator = custom_flow_from_directory(test_dir, (img_height, img_width), batch_size)


Found 4274 validated image filenames belonging to 2 classes.
Found 110 validated image filenames belonging to 2 classes.


In [ ]:
# Build a CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Calculate steps_per_epoch and validation_steps
steps_per_epoch_train = train_generator.samples // batch_size
steps_per_epoch_test = test_generator.samples // batch_size
epochs = 10
# Add 1 extra step if there are remaining samples not included in batches
if train_generator.samples % batch_size != 0:
    steps_per_epoch_train += 1
if test_generator.samples % batch_size != 0:
    steps_per_epoch_test += 1

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch_train,
    epochs=epochs,
    validation_data=test_generator,
    validation_steps=steps_per_epoch_test
)


Epoch 1/10
134/134 [==============================] - 1527s 11s/step - loss: 0.6336 - accuracy: 0.6273 - val_loss: 0.6633 - val_accuracy: 0.6455
Epoch 2/10
134/134 [==============================] - 565s 4s/step - loss: 0.5024 - accuracy: 0.7527 - val_loss: 0.6261 - val_accuracy: 0.7000
Epoch 3/10
134/134 [==============================] - 549s 4s/step - loss: 0.3752 - accuracy: 0.8346 - val_loss: 0.5098 - val_accuracy: 0.7273
Epoch 4/10
134/134 [==============================] - 543s 4s/step - loss: 0.2674 - accuracy: 0.8896 - val_loss: 0.4090 - val_accuracy: 0.7818
Epoch 5/10
134/134 [==============================] - 638s 5s/step - loss: 0.2181 - accuracy: 0.9160 - val_loss: 0.4708 - val_accuracy: 0.7909
Epoch 6/10
134/134 [==============================] - 550s 4s/step - loss: 0.1593 - accuracy: 0.9385 - val_loss: 0.4262 - val_accuracy: 0.8000
Epoch 7/10
134/134 [==============================] - 609s 5s/step - loss: 0.1274 - accuracy: 0.9518 - val_loss: 0.3859 - val_accuracy: 0.82